In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, precision_score, recall_score, f1_score

from sklearn.tree import DecisionTreeClassifier


In [3]:
train_df=pd.read_csv("/kaggle/input/titanic/train.csv")
test_df=pd.read_csv("/kaggle/input/titanic/test.csv")

In [4]:
train_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [5]:
train_df.info()
'''Lots of cabin have null values
    Age coloumn two has many null values
'''


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


'Lots of cabin have null values\n    Age coloumn two has many null values\n'

In [6]:
train_df['Survived'].value_counts()

Survived
0    549
1    342
Name: count, dtype: int64

In [7]:
train_df['Pclass']=train_df['Pclass'].astype(int)
test_df['Pclass']=test_df['Pclass'].astype(int)

#Baseline model

y_train_true= train_df['Survived']

#we are directly assuming all female survived
y_train_pred_gender_rule= train_df['Sex'].apply(lambda x:1 if x=='female' else 0)

def predict_survival_pclass_rule(row):
    if row['Sex'] == 'female' :
        return 1
    elif row['Sex'] == 'male' and row['Pclass']==1:
        return 1
    else:
        return 0
    return 0
        
y_train_pred_gender_pclass_rule= train_df.apply(predict_survival_pclass_rule,axis =1)


In [8]:
print('--------')
print('Training data evaluation -Comparing Non ML baseline models')

conf_mat_A = confusion_matrix(y_train_true, y_train_pred_gender_rule)
print('Confusion Matrix for Gender Rule')
print(conf_mat_A)

tn,fp,fn,tp = conf_mat_A.ravel()

print("Accuracy: ",accuracy_score(y_train_true, y_train_pred_gender_rule))
print("Precision: ",precision_score(y_train_true, y_train_pred_gender_rule))
print("Recall : ",recall_score(y_train_true, y_train_pred_gender_rule))
print("F1 score: ",f1_score(y_train_true, y_train_pred_gender_rule))

print('----B----')
conf_mat_B = confusion_matrix(y_train_true,y_train_pred_gender_pclass_rule)
print('Confusion Matrix for Gender Rule')
print(conf_mat_B)

tn,fp,fn,tp = conf_mat_B.ravel()

print("Accuracy: ",accuracy_score(y_train_true, y_train_pred_gender_pclass_rule))
print("Precision: ",precision_score(y_train_true, y_train_pred_gender_pclass_rule))
print("Recall : ",recall_score(y_train_true, y_train_pred_gender_pclass_rule))
print("F1 score: ",f1_score(y_train_true, y_train_pred_gender_pclass_rule))



--------
Training data evaluation -Comparing Non ML baseline models
Confusion Matrix for Gender Rule
[[468  81]
 [109 233]]
Accuracy:  0.7867564534231201
Precision:  0.7420382165605095
Recall :  0.6812865497076024
F1 score:  0.7103658536585366
----B----
Confusion Matrix for Gender Rule
[[391 158]
 [ 64 278]]
Accuracy:  0.7508417508417509
Precision:  0.6376146788990825
Recall :  0.8128654970760234
F1 score:  0.7146529562982005


In [9]:
print(y_train_true.shape)
print(train_df.shape)
print(y_train_pred_gender_pclass_rule.shape)

(891,)
(891, 12)
(891,)


In [10]:
y_test_pred_gender_pclass_rule= test_df.apply(predict_survival_pclass_rule,axis =1)

In [11]:
print(y_test_pred_gender_pclass_rule.shape)

(418,)


In [12]:
import pandas as pd

submission = pd.DataFrame()

submission['PassengerId'] = test_df['PassengerId']   # must exist in Titanic test set
submission['Survived'] = y_test_pred_gender_pclass_rule

submission.to_csv('submission.csv', index=False)


In [13]:
submission.shape

(418, 2)